In [1]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import powerlaw
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def readReview(path, business):
    review=[]
    reviewdf = pd.DataFrame.from_records(review)
    with open(path + "review.json", 'rb') as f:
        for l in f:
            data = json.loads(l)
            review.append(data)
            if len(review) > 1000000:
                df = pd.DataFrame.from_records(review)
                df = pd.merge(df, business, on=['business_id'])
                review = []
                reviewdf = pd.concat([reviewdf, df])
        df = pd.DataFrame.from_records(review)
        df = pd.merge(df, business, on=['business_id'])
        review = []
        reviewdf = pd.concat([reviewdf, df])
    return reviewdf

In [3]:
def readBusiness(path):
    business=[]
    for l in open(path + "business.json", 'rb').readlines():
        business.append(json.loads(l))
    businessdf = pd.DataFrame.from_records(business)
    return businessdf

In [4]:
path = "D:\study\\NU\\20winter\\396\yelp\\"
business = readBusiness(path)
business = business[business.state == "IL"]
business = business.drop(['stars'],axis=1)
review = readReview(path, business)

In [5]:
review['text length'] = review[['text']].applymap(lambda x: len(x.split(" ")))

transfer text into lower case

In [6]:
review['text1']=review['text'].apply(lambda sen:" ".join(x.lower() for x in sen.split()))
review[['text','text1']].head()

,text,text1
0,I got stuck in Myrtle Beach last week and stay...,i got stuck in myrtle beach last week and stay...
1,"Unfortunately, I have to agree with the other ...","unfortunately, i have to agree with the other ..."
2,Can't go wrong with 24 hour Starbuck's service...,can't go wrong with 24 hour starbuck's service...
3,I expected much more than what I received here...,i expected much more than what i received here...
4,This Courtyard is pretty on par with the other...,this courtyard is pretty on par with the other...


delete the punctuation in the text

In [7]:
review['text2']=review['text1'].str.replace('[^\w\s]','')
review[['text1','text2']].head()

,text1,text2
0,i got stuck in myrtle beach last week and stay...,i got stuck in myrtle beach last week and stay...
1,"unfortunately, i have to agree with the other ...",unfortunately i have to agree with the other n...
2,can't go wrong with 24 hour starbuck's service...,cant go wrong with 24 hour starbucks service w...
3,i expected much more than what i received here...,i expected much more than what i received here...
4,this courtyard is pretty on par with the other...,this courtyard is pretty on par with the other...


delete the stop words in text.

In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
review['text3']=review['text2'].apply(lambda sen:" ".join(x for x in sen.split() if x not in stop))
review[['text2','text3']].head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zxy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text2,text3
0,i got stuck in myrtle beach last week and stay...,got stuck myrtle beach last week stayed waitin...
1,unfortunately i have to agree with the other n...,unfortunately agree negative review thats post...
2,cant go wrong with 24 hour starbucks service w...,cant go wrong 24 hour starbucks service stayed...
3,i expected much more than what i received here...,expected much received first free internet pra...
4,this courtyard is pretty on par with the other...,courtyard pretty par newly decorated courtyard...


 Lemmatization

In [9]:
import nltk
nltk.download('wordnet')
from textblob import Word
review['text4'] = review['text3'].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
review[['text3','text4']]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zxy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text3,text4
0,got stuck myrtle beach last week stayed waitin...,got stuck myrtle beach last week stayed waitin...
1,unfortunately agree negative review thats post...,unfortunately agree negative review thats post...
2,cant go wrong 24 hour starbucks service stayed...,cant go wrong 24 hour starbucks service stayed...
3,expected much received first free internet pra...,expected much received first free internet pra...
4,courtyard pretty par newly decorated courtyard...,courtyard pretty par newly decorated courtyard...
...,...,...
4984,got fried chicken wings basically combo fried ...,got fried chicken wing basically combo fried o...
4985,deepfried chicken fish polish sausage wings fr...,deepfried chicken fish polish sausage wing fry...
4986,lovely bb amazingly comfortable beds gracious ...,lovely bb amazingly comfortable bed gracious h...
4987,beautiful little house friends wedding quite b...,beautiful little house friend wedding quite bi...


remove digit in the text.

In [10]:
 review['text5'] = review['text4'].str.replace(r'\d+','')

In [11]:
review[['text4','text5']].head()

,text4,text5
0,got stuck myrtle beach last week stayed waitin...,got stuck myrtle beach last week stayed waitin...
1,unfortunately agree negative review thats post...,unfortunately agree negative review thats post...
2,cant go wrong 24 hour starbucks service stayed...,cant go wrong hour starbucks service stayed m...
3,expected much received first free internet pra...,expected much received first free internet pra...
4,courtyard pretty par newly decorated courtyard...,courtyard pretty par newly decorated courtyard...


### NLP Classification Task

A score of 3 or more is considered positive, and marked as 1. A score of 1 or 2 is considered negative, and marked as -1.

In [12]:
def transfer(score):
    if score >= 3:
        return 1
    else:
        return -1

In [13]:
review['sentiment'] = review[['stars']].applymap(lambda x:transfer(x))

In [14]:
text=review['text5']
sentiment = review['sentiment']

MultinomialNB model predict sentiment

In [16]:
X = text
y = sentiment

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=3000,stop_words='english',lowercase=True)
transformer = TfidfTransformer()
X = transformer.fit_transform(vectorizer.fit_transform(X))

In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, f1_score
classifier = MultinomialNB()
classifier.fit(X, y)
classifier.coef_

array([[ -7.13789093,  -8.51720071,  -6.99086197, ...,  -8.81968593,
         -9.50244321, -11.75834379]])

In [19]:
from sklearn.model_selection import cross_val_score
CV_score = cross_val_score(MultinomialNB(), X, y, cv=5,scoring = 'accuracy')
CV_score

array([0.85616519, 0.86289086, 0.86299268, 0.87172528, 0.86333058])

In [20]:
CV_score.mean()

0.8634209175013522

LogisticRegression model predict sentiment

In [21]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(X, y)
model.coef_

d:\software\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([[ 2.05043997, -0.35160104, -0.47999705, ...,  0.89871575,
        -2.62525176, -1.825964  ]])

In [22]:
from sklearn.model_selection import cross_val_score
CV_score = cross_val_score(LogisticRegression(), X, y, cv=5,scoring = 'accuracy')
CV_score

d:\software\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.89380531, 0.8979351 , 0.89096059, 0.90028322, 0.90570046])

In [23]:
CV_score.mean()

0.8977369355687351

A score of 4 or 5 is considered positive, and marked as 1. A score of 1 or 2 is considered negative, and marked as -1. A score of 3 is considered natural, and marked as 0. 

In [16]:
def transfer(score):
    if score > 3:
        return 1
    elif score == 3:
        return 0
    else:
        return -1

In [17]:
review['sentiment'] = review[['stars']].applymap(lambda x:transfer(x))

In [18]:
text=review['text5']
sentiment = review['sentiment']
y_ = review['stars']

In [34]:
X = text
y = sentiment

MultinomialNB model predict sentiment

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=3000,stop_words='english',lowercase=True)
transformer = TfidfTransformer()
X = transformer.fit_transform(vectorizer.fit_transform(X))

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, f1_score
classifier = MultinomialNB()
classifier.fit(X, y)
classifier.coef_

array([[ -7.43125443,  -8.42560821,  -7.1419338 , ...,  -9.31437607,
         -7.39712371,  -7.95824117],
       [ -7.62113331,  -8.90269084,  -7.96494399, ...,  -8.29390329,
         -9.02205692, -10.1195099 ],
       [ -7.06343324,  -8.43075287,  -6.8665361 , ...,  -8.96458706,
         -9.57260266, -11.80566361]])

In [37]:
from sklearn.model_selection import cross_val_score
CV_score = cross_val_score(MultinomialNB(), X, y, cv=5,scoring = 'accuracy')
CV_score

array([0.76660767, 0.77120944, 0.77413264, 0.78251121, 0.77752862])

In [38]:
CV_score.mean()

0.7743979162499688

LogisticRegression model predict sentiment

In [39]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(X, y)
model.coef_

d:\software\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\software\python36\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


array([[-2.05043996,  0.35160105,  0.47999707, ..., -0.89871575,
         2.62525179,  1.82596404],
       [-0.83894206, -0.03995583, -2.04430618, ...,  1.05514656,
        -1.06914405, -1.04670731],
       [ 2.46209046, -0.41236365,  0.76320471, ..., -0.10719627,
        -2.16013661, -1.35867442]])

In [40]:
from sklearn.model_selection import cross_val_score
CV_score = cross_val_score(LogisticRegression(), X, y, cv=5,scoring = 'accuracy')
CV_score

d:\software\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\software\python36\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


array([0.81309735, 0.8160472 , 0.81307529, 0.82251593, 0.82296707])

In [41]:
CV_score.mean()

0.8175405669702409

LogisticRegression model predict stars

In [39]:
X_ = text
index_1 = y_==1
y_tmp1 = y_[index_1]
x_tmp1 = X_[index_1]
index_2 = y_==2
y_tmp2 = y_[index_2]
x_tmp2 = X_[index_2]
index_3 = y_==3
y_tmp3 = y_[index_3]
x_tmp3 = X_[index_3]
index_4 = y_==4
y_tmp4= y_[index_4]
x_tmp4 = X_[index_4]
X = pd.concat([X_,x_tmp1,x_tmp2,x_tmp2,x_tmp2,x_tmp3,x_tmp3,x_tmp4])
y = pd.concat([y_,y_tmp1,y_tmp2,y_tmp2,y_tmp2,y_tmp3,y_tmp3, y_tmp4])

In [42]:
X = text;
y = y_

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=3000,stop_words='english',lowercase=True)
transformer = TfidfTransformer()
X = transformer.fit_transform(vectorizer.fit_transform(X))

In [44]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(X, y)
model.coef_

d:\software\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\software\python36\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


array([[-1.86657172,  1.02722199,  0.93721474, ..., -0.53667165,
         2.53378716,  2.56594478],
       [-0.65381355, -0.7383874 , -0.97746696, ..., -0.41295469,
        -0.37588054, -1.36166977],
       [-0.83894202, -0.03995583, -2.04430614, ...,  1.05514658,
        -1.06914408, -1.04670718],
       [ 0.58246977, -0.9835633 , -1.83314412, ...,  0.33106331,
        -1.97971281, -0.98703067],
       [ 1.85069458,  0.49999522,  1.8098457 , ..., -0.26601579,
        -0.66651697, -0.95165568]])

In [45]:
from sklearn.model_selection import cross_val_score
CV_score = cross_val_score(LogisticRegression(), X, y, cv=5,scoring = 'accuracy')
CV_score

d:\software\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\software\python36\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


array([0.59981123, 0.59933931, 0.60273811, 0.60450844, 0.59931547])

In [46]:
CV_score.mean()

0.6011425126489289

MultinomialNB model predict sentiment

In [57]:
X = text;
y = y_

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=3000,stop_words='english',lowercase=True)
transformer = TfidfTransformer()
X = transformer.fit_transform(vectorizer.fit_transform(X))

In [61]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, f1_score
classifier = MultinomialNB()
classifier.fit(X, y)
classifier.coef_

array([[ -7.37308767,  -8.1238698 ,  -6.94854528, ...,  -9.41634549,
         -7.09012345,  -7.52804502],
       [ -7.57954535,  -9.03813547,  -7.61893118, ...,  -8.94283755,
         -8.26425672,  -9.54297798],
       [ -7.62113331,  -8.90269084,  -7.96494399, ...,  -8.29390329,
         -9.02205692, -10.1195099 ],
       [ -7.32271259,  -8.99150582,  -7.50104669, ...,  -8.54051851,
         -9.86280237, -10.89675016],
       [ -6.94430499,  -8.16967243,  -6.6188579 , ...,  -9.30077933,
         -9.30963473, -11.32669947]])

In [62]:
from sklearn.model_selection import cross_val_score
CV_score = cross_val_score(MultinomialNB(), X, y, cv=5,scoring = 'accuracy')
CV_score

array([0.55957999, 0.55969797, 0.56532515, 0.56284669, 0.56508911])

In [63]:
CV_score.mean()

0.5625077815677113